In [1]:
import sys
currentDir=%pwd
sys.path.append(currentDir)
import gipsy

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

from IPython.display import display, Image
from ipywidgets import interact, interactive, Layout
import ipywidgets as widgets
import traitlets
import ast

In [3]:
x = 12
y = 10


pH=7.4
#Insert uniprot Taxonomy ID
taxonomyID=83333
taxonomy_name="Escherichia coli \(strain K12"

csv_folder="Raw"
cut_folder="Cuts"
dbfile="Ecoli_Hres_db.csv"


In [5]:
df=pd.read_csv(dbfile)
df=gipsy.RowNormalizer(df)
index_list=df[df["num_pixels"]>= int(x*0.8)].index.values
df_plot=df[df["num_pixels"]>= int(x*0.8)].reset_index()
DFcoord=df_plot[df_plot.columns[df_plot.columns.get_loc("A1"):]]

letterindex=gipsy.IndexMaker(x,'l')                                                              
numbercols=np.arange(1,y+1)                                                                
sns.set(style="white")                                                                     
fig,ax = plt.subplots()  

slider= widgets.FloatSlider(value=0,min=0,max=1,step=0.01,discription="Threshold:")

selector= widgets.IntSlider(value=0,min=0,max=len(df_plot.index)-1,discription="Selector:")


b1 = widgets.Button(description ='next')
b2 = widgets.Button(description ='previous')
b3 = widgets.Button(description = 'reset')
b4 = widgets.Button(description ='save to file')

conc_options=['None']


for i in letterindex:
    if letterindex.index(i)+1 < len(letterindex):
        conc_options.append(i)
        conc_options.append(i+'/'+letterindex[letterindex.index(i)+1])
    else:
        conc_options.append(i)


cmdrop= widgets.SelectionSlider(options=conc_options,description='C_m:')
gradient_step=(8/(len(conc_options)-1))

ftext= widgets.FloatSlider(value=0,min=0,max=1,step=0.01,description="Threshold:")

mylink = traitlets.link((ftext, 'value'), (slider, 'value'))
butt=widgets.HBox([widgets.VBox([widgets.HBox([b2,b1]),widgets.HBox([b4,b3])]),widgets.VBox([ftext,cmdrop])])


def click_plus(name):
    df.set_value(index_list[selector.value],"Threshold",slider.value)
    if cmdrop.value !="None":
        df.set_value(index_list[selector.value],'Category',"useful")
    else:
        df.set_value(index_list[selector.value],'Category',"found")
        
    selector.value +=1
    if df.loc[index_list[selector.value],"Threshold"] != 0:
        slider.value=df.get_value(index_list[selector.value],"Threshold")
    else:
        slider.value=0
    if df.loc[index_list[selector.value],"[Urea]_50"] != 0:
        gradient_value=df.get_value(index_list[selector.value],"[Urea]_50")
        coord_local=int(gradient_value*(len(conc_options)/8)+gradient_step/2)
        cmdrop.value=conc_options[coord_local]
    else:
        cmdrop.value="None"
    
    
def click_minus(name):
    df.set_value(index_list[selector.value],"Threshold",slider.value)
    if cmdrop.value !="None":
        df.set_value(index_list[selector.value],'Category',"useful")
    else:
        df.set_value(index_list[selector.value],'Category',"found")
        
    selector.value -=1
    if df.loc[index_list[selector.value],"Threshold"] != 0:
        slider.value=df.get_value(index_list[selector.value],"Threshold")
    else:
        slider.value=0
    if df.loc[index_list[selector.value],"[Urea]_50"] != 0:
        gradient_value=df.get_value(index_list[selector.value],"[Urea]_50")
        coord_local=int(gradient_value*(len(conc_options)/8)+gradient_step/2)
        
        cmdrop.value=conc_options[coord_local]
    else:
        cmdrop.value="None"
    
    
        

def add_Thold(Thold):
    df.set_value(index_list[selector.value],"Threshold",slider.value)

def file_saver(name):
    th_dbfile="Th_"+dbfile
    df_th = gipsy.Smear_check_th(df,y)
    df_th.to_csv(th_dbfile,index=False)
    
def cm_calc(name):
    coord_int=conc_options.index(cmdrop.value)
    if coord_int > 0:
        df.loc[index_list[selector.value],"[Urea]_50"]= gradient_step*coord_int-gradient_step/2
    else:
        df.loc[index_list[selector.value],"[Urea]_50"]=0
    
def resetting(name):
    slider.value=0
    cmdrop.value='None'
    
b1.on_click(click_plus)
b2.on_click(click_minus)
b3.on_click(resetting)
b4.on_click(file_saver)
cmdrop.observe(cm_calc)


slider.value=df.loc[index_list[selector.value],"Threshold"]


def plotter(row,Thold):
    dfr=DFcoord.loc[row]
    rsdata=dfr.values.reshape((x,y)).T
    df_rs=pd.DataFrame(rsdata,columns=letterindex,index=numbercols)
    
    mask_below=df_rs <= Thold   
    mask_above=df_rs > Thold
    mask_zero=df_rs == 0
    mask_added=mask_above | mask_zero
    mask_subbed=mask_below ^ mask_zero
    
    sns_plot=sns.heatmap(df_rs,cmap="Blues",mask=mask_subbed,annot=True,cbar=True)
    sns_plot2=sns.heatmap(df_rs,mask = mask_added,cmap="Reds_r",annot=True,cbar=False)
    sns_plot.xaxis.tick_top()
    pname=(df.loc[index_list[row],"prot_desc"]).split(' GN=')[0]  
    print("Protein: %s of %s"%(selector.value+1,len(df_plot)))
    print(pname)
    plt.show()
pter=interactive(plotter,row=selector,Thold=slider)    
slider.layout.visibility = 'hidden'
selector.layout.visibility ='hidden'
display(butt,pter)

In [4]:
df_th=pd.read_csv('Th_'+dbfile)
df_th

,index,prot_acc,prot_desc,prot_mass,pI,[Urea]_50,Delta_G,Category,num_pixels,Smear,...,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10
0,0,3MG2_ECOLI,DNA-3-methyladenine glycosylase 2 OS=Escherich...,31601,7.482822,0.000000,0,found,1,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,1,3PASE_ECOLI,Inorganic triphosphatase OS=Escherichia coli (...,48530,5.499170,0.000000,0,found,15,False,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,4,6PGD_ECOLI,"6-phosphogluconate dehydrogenase, decarboxylat...",51563,4.952432,0.000000,0,found,55,True,...,0.000000,0.180769,0.143590,0.234615,0.061538,0.000000,0.000000,0.0,0.012821,0.000000
3,7,AAT_ECOLI,Aspartate aminotransferase OS=Escherichia coli...,43831,5.384736,0.000000,0,found,61,True,...,0.000000,0.394794,0.308026,0.086768,0.000000,0.000000,0.158351,0.0,0.000000,0.000000
4,8,ABDH_ECOBW,Gamma-aminobutyraldehyde dehydrogenase OS=Esch...,51197,5.435596,0.000000,0,found,26,False,...,0.000000,0.208333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,9,ACCA_ECOBW,Acetyl-coenzyme A carboxylase carboxyl transfe...,35333,5.600889,0.000000,0,found,14,False,...,0.051948,0.107143,0.000000,0.048701,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,10,ACCC_ECOLI,Biotin carboxylase OS=Escherichia coli (strain...,49745,6.338350,0.000000,0,found,26,False,...,0.215116,0.209302,0.127907,0.000000,0.000000,0.000000,0.058140,0.0,0.000000,0.000000
7,11,ACCD_ECODH,Acetyl-coenzyme A carboxylase carboxyl transfe...,33642,6.912295,0.000000,0,found,20,False,...,0.000000,0.315315,0.135135,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,12,ACEA_ECOLI,Isocitrate lyase OS=Escherichia coli (strain K...,47777,5.041436,0.000000,0,found,83,True,...,0.058559,0.282282,0.223724,0.226727,0.036036,0.000000,0.033033,0.0,0.000000,0.000000
9,13,ACKA_ECOLI,Acetate kinase OS=Escherichia coli (strain K12...,43605,5.588174,0.000000,0,found,70,True,...,0.063725,0.127451,0.122549,0.196078,0.127451,0.000000,0.000000,0.0,0.000000,0.000000


In [13]:
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

fig=plt.subplots(figsize=(15,10))

ncount=len(df_th)

ax2=plt.subplot(221)
ax3=plt.subplot(212)
ax1=plt.subplot(222)


p_left=sns.countplot(df_th["Category"],
                      ax=ax2,
                      order=["found","useful"],
                      palette=sns.color_palette("husl",3))


p_right=sns.countplot(df_th["num_pixels"],hue=df_th["Category"],
                      hue_order=["found","useful"],
                      ax=ax3,
                      palette=sns.color_palette("husl",3))

p_first=sns.countplot(df_th["Smear"],
                      ax=ax1,
                      palette=sns.color_palette("hls",2))


for p in p_left.patches:
    px=p.get_bbox().get_points()[:,0]
    py=p.get_bbox().get_points()[1,1]
    p_left.annotate('{:.1f}%\n(n={:.0f})'.format((100.*py)/ncount,py), (px.mean(), py+5), 
            ha='center', va='bottom',size=15)
    
for p in p_first.patches:
    px=p.get_bbox().get_points()[:,0]
    py=p.get_bbox().get_points()[1,1]
    p_first.annotate('{:.1f}%\n(n={:d})'.format(100.*py/ncount,int(py)), (px.mean(), py*0.5), 
            ha='center', va='center',size=15,color='white')

usef=mpatches.Patch(color=sns.color_palette("husl",3)[1], label="Useful")
fnd=mpatches.Patch(color=sns.color_palette("husl",3)[0], label="Found")
Nsmear=mpatches.Patch(color=sns.color_palette("hls",2)[0], label="No Smear")
Smear=mpatches.Patch(color=sns.color_palette("hls",2)[1], label="Smear")

ax2.axhline(y=0,color='grey') 
ax3.axhline(y=0,color='grey')
ax1.axhline(y=0,color='grey')

ax2.legend(handles=[fnd,usef],loc=0,fontsize=25,frameon=True,facecolor='white')
ax3.legend(handles=[fnd,usef],loc=9,ncol=3,fontsize=25,frameon=True,facecolor='white')
ax1.legend(handles=[Nsmear,Smear],loc=0,fontsize=25,frameon=True,facecolor='white')


ax2.axes.get_xaxis().set_visible(False)
ax1.axes.get_xaxis().set_visible(False)


p_left.set_ylim(0,ncount*1.20)
p_right.set_ylabel("Protein Count",size=20)
p_left.set_ylabel("Protein Count",size=20)
p_first.set_ylabel("Protein Count",size=20)
p_right.set_xlabel("Number of active Pixels",size=20)

ax1.yaxis.set_tick_params(labelsize=16)
ax2.yaxis.set_tick_params(labelsize=16)
ax3.yaxis.set_tick_params(labelsize=16)
plt.setp(ax3.get_xticklabels(), fontsize=10, rotation=60)


ax2.set_title("Categories - post editor",size=36,y=1.05)
ax3.set_title("Distribution of active pixels - post editor",size=36,y=1.05)
ax1.set_title("Smear distribution - post editor",size=36,y=1.05)
plt.tight_layout()
#plt.show()
plt.savefig("%s_edited_category.png"%dbfile.split('_db')[0],dpi=500)
plt.close()

In [23]:
df_useful=df_th[df_th["Category"]=='useful']
sns.set(font_scale=1.3)
fig,ax = plt.subplots(1,1,figsize=(8,5))

gradient_convert=(8/((y*2)-1))
gradient_range=[]

for index,row in df_useful.iterrows():
    Cm_value=df_useful.loc[index,"[Urea]_50"]
    low=Cm_value
    high=Cm_value+gradient_convert
    cont='%s-%s'%(np.round(low,2),np.round(high,2))
    gradient_range.append(cont)
    
df_useful["[Urea]_50"]=gradient_range
order=sorted(set(gradient_range))
conuter= sns.countplot(df_useful["[Urea]_50"],order=order,ax=ax)


for p in conuter.patches:
    px=p.get_bbox().get_points()[:,0]
    py=p.get_bbox().get_points()[1,1]
    conuter.annotate('{:.1f}%\n(n={:.0f})'.format((100.*py)/len(df_useful),py), (px.mean(), py+0.1), 
            ha='center', va='bottom',size=10)


conuter.set_ylabel("Protein Count",size=18)
conuter.set_xlabel("Ranges of [Urea]$^{50\%}$",size=18)

plt.setp(ax.get_xticklabels(), fontsize=10, rotation=60)

conuter.set_title("Distribution of [Urea]$^{50\%}$",size=25,y=1.05)
conuter.axhline(y=0,color='grey')
conuter.set_ylim(0,17)

plt.tight_layout()
plt.savefig("%s_Cm_dist.png"%dbfile.split('_db')[0],dpi=500)
plt.close()

/home/kaslangg/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
DFcoord=df_th[df_th.columns[df_th.columns.get_loc("A1"):]]
df_threshold=df_th["Threshold"]
letterindex=gipsy.IndexMaker(x,'l')                                                              
numbercols=np.arange(1,y+1)                                                                
sns.set(style="white")                                                                     
  

if not os.path.isdir("heatmaps"):
        os.makedirs("heatmaps")

for index,row in DFcoord.iterrows():
    fig,ax = plt.subplots()
    rsdata=row.values.reshape((x,y)).T
    df_rs=pd.DataFrame(rsdata,columns=letterindex,index=numbercols)
    th_value=df_threshold[index]
    mask_below=df_rs <= th_value  
    mask_above=df_rs > th_value
    mask_zero=df_rs == 0
    mask_added=mask_above | mask_zero
    mask_subbed=mask_below ^ mask_zero
    
    sns_plot=sns.heatmap(df_rs,cmap="Blues",mask=mask_subbed,annot=True,cbar=True)
    sns_plot2=sns.heatmap(df_rs,mask = mask_added,cmap="Reds_r",annot=True,cbar=False)
    sns_plot.xaxis.tick_top()
    pname=(df_th.loc[index,"prot_desc"]).split(' OS=')
    if len(pname) > 1:
        ax.set_title("%s\nOS=%s"%(pname[0],pname[1]),y=1.1,size=15)
        savefilename= os.getcwd()+"/heatmaps/%s.png"%pname[0]
    else:
        ax.set_title("%s"%(pname),y=1.1,size=15)
        savefilename= os.getcwd()+"/heatmaps/%s.png"%pname
    plt.tight_layout()
    plt.savefig(savefilename,dpi=500)
    plt.close()
    plt.clf()

In [19]:
df=pd.read_csv('Th_'+dbfile)

In [22]:
df_print=df[["prot_acc","prot_desc","prot_mass","pI","Category","[Urea]_50","Threshold","num_pixels","Smear"]]

/home/kaslangg/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
df_print.to_csv("Ecoli_Hres_pList.csv",index=False)